In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7261,2021-02-03T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1317,193,1510,39612,...,NaN,7.0,NaN,NaN,138739.0,0.0,1534086.0,329507.0,ITG,ITG1
7262,2021-02-03T17:00:00,ITA,9,Toscana,43.769231,11.255889,663,109,772,8573,...,NaN,4.0,NaN,NaN,135658.0,121.0,2175251.0,101680.0,ITI,ITI1
7263,2021-02-03T17:00:00,ITA,10,Umbria,43.106758,12.388247,367,64,431,5568,...,Si fa presente come un ricovero (in terapia in...,6.0,NaN,NaN,36935.0,0.0,604971.0,41746.0,ITI,ITI2
7264,2021-02-03T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,29,3,32,157,...,NaN,0.0,NaN,NaN,7815.0,0.0,69748.0,1134.0,ITC,ITC2


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note', 'note_test', 'note_casi', 'codice_nuts_1', 'codice_nuts_2']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7261,2021-02-03T17:00:00,19,Sicilia,1317,193,1510,39612,41122,-491,886,...,NaN,NaN,138739,1863593,1002162.0,7.0,138739.0,0.0,1534086.0,329507.0
7262,2021-02-03T17:00:00,9,Toscana,663,109,772,8573,9345,143,551,...,NaN,NaN,135779,2276931,1213838.0,4.0,135658.0,121.0,2175251.0,101680.0
7263,2021-02-03T17:00:00,10,Umbria,367,64,431,5568,5999,227,458,...,NaN,NaN,36935,646717,286806.0,6.0,36935.0,0.0,604971.0,41746.0
7264,2021-02-03T17:00:00,2,Valle d'Aosta,29,3,32,157,189,-12,11,...,NaN,NaN,7815,70882,41731.0,0.0,7815.0,0.0,69748.0,1134.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-02-03', '2021-02-02')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-02-03T17:00:00,1,Piemonte,2092,147,2239,9813,12052,-73,819,...,NaN,NaN,225195,2122420,1113483.0,11.0,222941.0,2254.0,1901896.0,220524.0
1,2021-02-03T17:00:00,2,Valle d'Aosta,29,3,32,157,189,-12,11,...,NaN,NaN,7815,70882,41731.0,0.0,7815.0,0.0,69748.0,1134.0
2,2021-02-03T17:00:00,3,Lombardia,3554,359,3913,43432,47345,786,1738,...,NaN,NaN,541797,5743914,2856882.0,20.0,540016.0,1781.0,5586597.0,157317.0
3,2021-02-03T17:00:00,5,Veneto,1306,176,1482,28878,30360,-1684,629,...,NaN,NaN,313945,4243554,1329843.0,17.0,312557.0,1388.0,3779065.0,464489.0
4,2021-02-03T17:00:00,6,Friuli Venezia Giulia,561,63,624,10556,11180,-46,692,...,NaN,NaN,68613,1166449,422735.0,5.0,65818.0,2795.0,1126574.0,39875.0
5,2021-02-03T17:00:00,7,Liguria,641,64,705,3734,4439,54,233,...,NaN,NaN,70288,876302,381711.0,5.0,70288.0,0.0,829405.0,46897.0
6,2021-02-03T17:00:00,8,Emilia-Romagna,2122,194,2316,42086,44402,-910,1047,...,NaN,NaN,221615,3166204,1369671.0,8.0,221556.0,59.0,3024793.0,141411.0
7,2021-02-03T17:00:00,9,Toscana,663,109,772,8573,9345,143,551,...,NaN,NaN,135779,2276931,1213838.0,4.0,135658.0,121.0,2175251.0,101680.0
8,2021-02-03T17:00:00,10,Umbria,367,64,431,5568,5999,227,458,...,NaN,NaN,36935,646717,286806.0,6.0,36935.0,0.0,604971.0,41746.0
9,2021-02-03T17:00:00,11,Marche,535,68,603,7750,8353,-328,408,...,NaN,NaN,56417,702605,423244.0,4.0,56417.0,0.0,673309.0,29296.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-02-02T17:00:00,1,Piemonte,2138,152,2290,9835,12125,-139,765,...,NaN,NaN,224376,2103721,1107431.0,4.0,222262.0,2114.0,1894771.0,208950.0
1,2021-02-02T17:00:00,2,Valle d'Aosta,25,3,28,173,201,-28,4,...,NaN,NaN,7804,70530,41648.0,0.0,7804.0,0.0,69464.0,1066.0
2,2021-02-02T17:00:00,3,Lombardia,3544,361,3905,42654,46559,-1916,912,...,NaN,NaN,540059,5705263,2847233.0,21.0,538440.0,1619.0,5558207.0,147056.0
3,2021-02-02T17:00:00,5,Veneto,1423,193,1616,30428,32044,-1776,621,...,NaN,NaN,313316,4204507,1326953.0,15.0,311850.0,1466.0,3766253.0,438254.0
4,2021-02-02T17:00:00,6,Friuli Venezia Giulia,579,61,640,10586,11226,-83,258,...,NaN,NaN,67921,1154752,418885.0,8.0,65319.0,2602.0,1118072.0,36680.0
5,2021-02-02T17:00:00,7,Liguria,630,64,694,3691,4385,-181,238,...,NaN,NaN,70055,869782,380164.0,2.0,70055.0,0.0,825622.0,44160.0
6,2021-02-02T17:00:00,8,Emilia-Romagna,2176,198,2374,42938,45312,-1295,879,...,NaN,NaN,220575,3142398,1365345.0,9.0,220517.0,58.0,3011257.0,131141.0
7,2021-02-02T17:00:00,9,Toscana,669,109,778,8424,9202,-80,399,...,NaN,NaN,135228,2259345,1205264.0,18.0,135126.0,102.0,2164743.0,94602.0
8,2021-02-02T17:00:00,10,Umbria,369,61,430,5342,5772,129,283,...,NaN,NaN,36477,638694,285288.0,9.0,36477.0,0.0,600641.0,38053.0
9,2021-02-02T17:00:00,11,Marche,544,68,612,8069,8681,-491,333,...,NaN,NaN,56009,697600,420206.0,2.0,56009.0,0.0,669557.0,28043.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-02-03T17:00:00,1,Piemonte,2092,147,2239,9813,12052,-73,819,...,NaN,NaN,225195,2122420,1113483.0,11.0,222941.0,2254.0,1901896.0,220524.0
1,2021-02-03T17:00:00,2,Valle d'Aosta,29,3,32,157,189,-12,11,...,NaN,NaN,7815,70882,41731.0,0.0,7815.0,0.0,69748.0,1134.0
2,2021-02-03T17:00:00,3,Lombardia,3554,359,3913,43432,47345,786,1738,...,NaN,NaN,541797,5743914,2856882.0,20.0,540016.0,1781.0,5586597.0,157317.0
3,2021-02-03T17:00:00,4,Trentino-Alto Adige,399,65,464,7781,8245,841,914,...,0.0,0.0,69788,1054134,345361.0,2.0,62279.0,7509.0,947667.0,106467.0
4,2021-02-03T17:00:00,5,Veneto,1306,176,1482,28878,30360,-1684,629,...,NaN,NaN,313945,4243554,1329843.0,17.0,312557.0,1388.0,3779065.0,464489.0
5,2021-02-03T17:00:00,6,Friuli Venezia Giulia,561,63,624,10556,11180,-46,692,...,NaN,NaN,68613,1166449,422735.0,5.0,65818.0,2795.0,1126574.0,39875.0
6,2021-02-03T17:00:00,7,Liguria,641,64,705,3734,4439,54,233,...,NaN,NaN,70288,876302,381711.0,5.0,70288.0,0.0,829405.0,46897.0
7,2021-02-03T17:00:00,8,Emilia-Romagna,2122,194,2316,42086,44402,-910,1047,...,NaN,NaN,221615,3166204,1369671.0,8.0,221556.0,59.0,3024793.0,141411.0
8,2021-02-03T17:00:00,9,Toscana,663,109,772,8573,9345,143,551,...,NaN,NaN,135779,2276931,1213838.0,4.0,135658.0,121.0,2175251.0,101680.0
9,2021-02-03T17:00:00,10,Umbria,367,64,431,5568,5999,227,458,...,NaN,NaN,36935,646717,286806.0,6.0,36935.0,0.0,604971.0,41746.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,0,2021-02-02T17:00:00,1,Piemonte,2138,152,2290,9835,12125,-139,...,NaN,NaN,224376,2103721,1107431.0,4.0,222262.0,2114.0,1894771.0,208950.0
1,1,2021-02-02T17:00:00,2,Valle d'Aosta,25,3,28,173,201,-28,...,NaN,NaN,7804,70530,41648.0,0.0,7804.0,0.0,69464.0,1066.0
2,2,2021-02-02T17:00:00,3,Lombardia,3544,361,3905,42654,46559,-1916,...,NaN,NaN,540059,5705263,2847233.0,21.0,538440.0,1619.0,5558207.0,147056.0
3,19,2021-02-02T17:00:00,4,Trentino-Alto Adige,407,68,475,6929,7404,122,...,0.0,0.0,68874,1041992,344217.0,2.0,61927.0,6947.0,942313.0,99679.0
4,3,2021-02-02T17:00:00,5,Veneto,1423,193,1616,30428,32044,-1776,...,NaN,NaN,313316,4204507,1326953.0,15.0,311850.0,1466.0,3766253.0,438254.0
5,4,2021-02-02T17:00:00,6,Friuli Venezia Giulia,579,61,640,10586,11226,-83,...,NaN,NaN,67921,1154752,418885.0,8.0,65319.0,2602.0,1118072.0,36680.0
6,5,2021-02-02T17:00:00,7,Liguria,630,64,694,3691,4385,-181,...,NaN,NaN,70055,869782,380164.0,2.0,70055.0,0.0,825622.0,44160.0
7,6,2021-02-02T17:00:00,8,Emilia-Romagna,2176,198,2374,42938,45312,-1295,...,NaN,NaN,220575,3142398,1365345.0,9.0,220517.0,58.0,3011257.0,131141.0
8,7,2021-02-02T17:00:00,9,Toscana,669,109,778,8424,9202,-80,...,NaN,NaN,135228,2259345,1205264.0,18.0,135126.0,102.0,2164743.0,94602.0
9,8,2021-02-02T17:00:00,10,Umbria,369,61,430,5342,5772,129,...,NaN,NaN,36477,638694,285288.0,9.0,36477.0,0.0,600641.0,38053.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione
0,2021-02-03T17:00:00,1,Piemonte,2092,147,2239,9813,12052,-73,819,...,NaN,225195,2122420,1113483.0,11.0,222941.0,2254.0,1901896.0,220524.0,4341375
1,2021-02-03T17:00:00,2,Valle d'Aosta,29,3,32,157,189,-12,11,...,NaN,7815,70882,41731.0,0.0,7815.0,0.0,69748.0,1134.0,125501
2,2021-02-03T17:00:00,3,Lombardia,3554,359,3913,43432,47345,786,1738,...,NaN,541797,5743914,2856882.0,20.0,540016.0,1781.0,5586597.0,157317.0,10103969
3,2021-02-03T17:00:00,4,Trentino-Alto Adige,399,65,464,7781,8245,841,914,...,0.0,69788,1054134,345361.0,2.0,62279.0,7509.0,947667.0,106467.0,1074819
4,2021-02-03T17:00:00,5,Veneto,1306,176,1482,28878,30360,-1684,629,...,NaN,313945,4243554,1329843.0,17.0,312557.0,1388.0,3779065.0,464489.0,4907704
5,2021-02-03T17:00:00,6,Friuli Venezia Giulia,561,63,624,10556,11180,-46,692,...,NaN,68613,1166449,422735.0,5.0,65818.0,2795.0,1126574.0,39875.0,1211357
6,2021-02-03T17:00:00,7,Liguria,641,64,705,3734,4439,54,233,...,NaN,70288,876302,381711.0,5.0,70288.0,0.0,829405.0,46897.0,1543127
7,2021-02-03T17:00:00,8,Emilia-Romagna,2122,194,2316,42086,44402,-910,1047,...,NaN,221615,3166204,1369671.0,8.0,221556.0,59.0,3024793.0,141411.0,4467118
8,2021-02-03T17:00:00,9,Toscana,663,109,772,8573,9345,143,551,...,NaN,135779,2276931,1213838.0,4.0,135658.0,121.0,2175251.0,101680.0,3722729
9,2021-02-03T17:00:00,10,Umbria,367,64,431,5568,5999,227,458,...,NaN,36935,646717,286806.0,6.0,36935.0,0.0,604971.0,41746.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione,totale_posti_terapia_intensiva
0,2021-02-03T17:00:00,1,Piemonte,2092,147,2239,9813,12052,-73,819,...,225195,2122420,1113483.0,11.0,222941.0,2254.0,1901896.0,220524.0,4341375,575
1,2021-02-03T17:00:00,2,Valle d'Aosta,29,3,32,157,189,-12,11,...,7815,70882,41731.0,0.0,7815.0,0.0,69748.0,1134.0,125501,20
2,2021-02-03T17:00:00,3,Lombardia,3554,359,3913,43432,47345,786,1738,...,541797,5743914,2856882.0,20.0,540016.0,1781.0,5586597.0,157317.0,10103969,1036
3,2021-02-03T17:00:00,4,Trentino-Alto Adige,399,65,464,7781,8245,841,914,...,69788,1054134,345361.0,2.0,62279.0,7509.0,947667.0,106467.0,1074819,106
4,2021-02-03T17:00:00,5,Veneto,1306,176,1482,28878,30360,-1684,629,...,313945,4243554,1329843.0,17.0,312557.0,1388.0,3779065.0,464489.0,4907704,1016
5,2021-02-03T17:00:00,6,Friuli Venezia Giulia,561,63,624,10556,11180,-46,692,...,68613,1166449,422735.0,5.0,65818.0,2795.0,1126574.0,39875.0,1211357,180
6,2021-02-03T17:00:00,7,Liguria,641,64,705,3734,4439,54,233,...,70288,876302,381711.0,5.0,70288.0,0.0,829405.0,46897.0,1543127,209
7,2021-02-03T17:00:00,8,Emilia-Romagna,2122,194,2316,42086,44402,-910,1047,...,221615,3166204,1369671.0,8.0,221556.0,59.0,3024793.0,141411.0,4467118,634
8,2021-02-03T17:00:00,9,Toscana,663,109,772,8573,9345,143,551,...,135779,2276931,1213838.0,4.0,135658.0,121.0,2175251.0,101680.0,3722729,523
9,2021-02-03T17:00:00,10,Umbria,367,64,431,5568,5999,227,458,...,36935,646717,286806.0,6.0,36935.0,0.0,604971.0,41746.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-02-03T17:00:00,3,Lombardia,3554,359,3913,43432,47345,786,1738,...,34.65,778,826,38651,9649.0,46,906,4.50,0.01720,5.36
1,2021-02-03T17:00:00,15,Campania,1475,100,1575,60542,62117,258,1539,...,19.80,266,620,19429,11245.0,17,1264,7.92,0.02660,3.90
2,2021-02-03T17:00:00,12,Lazio,2350,279,2629,57140,59769,-1004,1164,...,32.94,-1008,322,27558,18120.0,49,2119,4.22,0.01984,3.55
3,2021-02-03T17:00:00,8,Emilia-Romagna,2122,194,2316,42086,44402,-910,1047,...,30.60,-852,168,23806,4326.0,76,1874,4.40,0.02344,4.96
4,2021-02-03T17:00:00,16,Puglia,1429,155,1584,50131,51715,13,1044,...,42.01,31,165,10793,4273.0,46,985,9.67,0.02605,3.12
5,2021-02-03T17:00:00,4,Trentino-Alto Adige,399,65,464,7781,8245,841,914,...,61.32,852,103,12142,1144.0,8,65,7.53,0.08504,6.49
6,2021-02-03T17:00:00,19,Sicilia,1317,193,1510,39612,41122,-491,886,...,32.82,-472,-98,24130,6240.0,34,1343,3.67,0.01783,2.79
7,2021-02-03T17:00:00,1,Piemonte,2092,147,2239,9813,12052,-73,819,...,25.57,-22,54,18699,6052.0,25,867,4.38,0.01886,5.19
8,2021-02-03T17:00:00,6,Friuli Venezia Giulia,561,63,624,10556,11180,-46,692,...,35.00,-30,434,11697,3850.0,27,711,5.92,0.05713,5.66
9,2021-02-03T17:00:00,5,Veneto,1306,176,1482,28878,30360,-1684,629,...,17.32,-1550,8,39047,2890.0,57,2256,1.61,0.01282,6.40


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,65,41,106,61.32
1,Umbria,64,53,117,54.70
2,Marche,68,75,143,47.55
3,Puglia,155,214,369,42.01
4,Friuli Venezia Giulia,63,117,180,35.00
5,Lombardia,359,677,1036,34.65
6,Lazio,279,568,847,32.94
7,Sicilia,193,395,588,32.82
8,Abruzzo,47,101,148,31.76
9,Liguria,64,145,209,30.62


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,2021-02-01T17:00:00,22,P.A. Trento,180,35,215,2288,2503,34,53,...,NaN,NaN,27580,529694,161795.0,1.0,25486.0,2094.0,511445.0,18249.0
688,2021-02-02T17:00:00,21,P.A. Bolzano,233,34,267,4741,5008,229,639,...,NaN,NaN,41122,509703,182033.0,0.0,36389.0,4733.0,429532.0,80171.0
689,2021-02-02T17:00:00,22,P.A. Trento,174,34,208,2188,2396,-107,172,...,NaN,NaN,27752,532289,162184.0,2.0,25538.0,2214.0,512781.0,19508.0
690,2021-02-03T17:00:00,21,P.A. Bolzano,241,32,273,5477,5750,742,653,...,NaN,NaN,41775,518555,182952.0,0.0,36703.0,5072.0,432893.0,85662.0


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,2021-01-30T17:00:00,4,Trentino-Alto Adige,389,67,456,17340,17796,563,870,...,9.0,10953.0,1022.0,577.0,14.69,2.56,7.94,85.13,4.69,3.27
342,2021-01-31T17:00:00,4,Trentino-Alto Adige,401,67,468,6619,7087,-10709,602,...,5.0,7324.0,1676.0,-10721.0,14.32,6.60,8.22,35.92,0.00,-60.18
343,2021-02-01T17:00:00,4,Trentino-Alto Adige,413,69,482,6800,7282,195,369,...,2.0,4025.0,1284.0,181.0,14.32,6.62,9.17,28.74,2.99,2.75
344,2021-02-02T17:00:00,4,Trentino-Alto Adige,407,68,475,6929,7404,122,811,...,8.0,11746.0,1065.0,129.0,14.32,6.42,6.90,76.15,-1.45,1.68


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
